<a href="https://colab.research.google.com/github/vnshtiwari/pbdata/blob/main/Insurance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install langchain-community
!pip install langchain_openai
!pip install pymupdf
!pip install chromadb

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
# from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
import openai
from dotenv import load_dotenv
import os
import shutil
import fitz
from langchain.embeddings import HuggingFaceEmbeddings


In [ ]:
!wget https://healthstatic.policybazaar.com/health-insurance/Information/Care_Health/CARE%20SUPREME%20PROSPECTUS.pdf -O /content/Prospectus.pdf
!wget https://healthstatic.policybazaar.com/health-insurance/Information/Care_Health/TandC/care-supreme---policy-terms-and-conditions.pdf -O /content/Policy_wording.pdf
# Open the PDF file
pdf_paths = ["/content/Policy_wording.pdf", "/content/Prospectus.pdf"]
documents = []
for pdf_path in pdf_paths:
  doc = fitz.open(pdf_path)

  # Extract text from all pages
  for page_num in range(doc.page_count):
      page = doc.load_page(page_num)  # Load each page
      text = page.get_text()  # Extract text from the page

      # Create a Document object for each page
      document = Document(page_content=text, metadata={"page_num": page_num + 1})
      documents.append(document)

  # Split the text into chunks
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True)

chunks = text_splitter.split_documents(documents)
print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

--2025-07-17 18:43:14--  https://healthstatic.policybazaar.com/health-insurance/Information/Care_Health/CARE%20SUPREME%20PROSPECTUS.pdf
Resolving healthstatic.policybazaar.com (healthstatic.policybazaar.com)... 18.172.134.74, 18.172.134.114, 18.172.134.115, ...
Connecting to healthstatic.policybazaar.com (healthstatic.policybazaar.com)|18.172.134.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 249374 (244K) [application/pdf]
Saving to: ‘/content/Prospectus.pdf’

/content/Prospectus 100%[===================>] 243.53K  --.-KB/s    in 0.05s   

2025-07-17 18:43:14 (5.02 MB/s) - ‘/content/Prospectus.pdf’ saved [249374/249374]

--2025-07-17 18:43:14--  https://healthstatic.policybazaar.com/health-insurance/Information/Care_Health/TandC/care-supreme---policy-terms-and-conditions.pdf
Resolving healthstatic.policybazaar.com (healthstatic.policybazaar.com)... 18.172.134.74, 18.172.134.114, 18.172.134.115, ...
Connecting to healthstatic.policybazaar.com (healthstatic

In [ ]:
import os

CHROMA_PATH = os.path.join(os.getcwd(), "my_chroma_db")

In [ ]:

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

db = Chroma.from_documents(
    chunks,
    embedding,
    persist_directory=CHROMA_PATH
)

/tmp/ipython-input-5-3691599028.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
query = "Coverage"
results = db.similarity_search(query, k=3)  # get top 3 most similar chunks

for i, doc in enumerate(results):
    print(f"\nResult {i+1}:\n{doc.page_content}")

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from langchain.prompts import ChatPromptTemplate

# Load the model and tokenizer once (globally)
model_name = "mistralai/Mistral-7B-Instruct-v0.1"  # Replace with your model

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"  # Automatically use GPU if available
)

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

def answer_question(question):
    # Retrieve context based on the question
    results = db.similarity_search_with_relevance_scores(question, k=3)
    print(results)

    # Format the prompt with the top search results
    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=question)

    # Tokenize and run through model
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=300, do_sample=True, temperature=0.7)
    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract response text if the model returns the full prompt + answer
    if prompt.strip() in response_text:
        response_text = response_text.split(prompt.strip())[-1].strip()

    sources = [doc.metadata.get("source", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    return formatted_response

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
answer_question("Coverage")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[(Document(metadata={'start_index': 1888, 'page_num': 5}, page_content='coverage has elapsed as specified, since the inception of the first Policy with us.\n \nNote: The above Optional Benefit can be opted only if this policy is issued for the first time with us.\n3.2.6 Optional Benefit : Deductible'), 0.3061251376097507), (Document(metadata={'start_index': 1888, 'page_num': 5}, page_content='coverage has elapsed as specified, since the inception of the first Policy with us.\n \nNote: The above Optional Benefit can be opted only if this policy is issued for the first time with us.\n3.2.6 Optional Benefit : Deductible'), 0.3061251376097507), (Document(metadata={'start_index': 1888, 'page_num': 5}, page_content='coverage has elapsed as specified, since the inception of the first Policy with us.\n \nNote: The above Optional Benefit can be opted only if this policy is issued for the first time with us.\n3.2.6 Optional Benefit : Deductible'), 0.3061251376097507)]
